The news data used for sentiment analysis is only available for the last 30 days. The news data is collected from a free news API, and includes articles related to the company associated with the ticker symbol. The news articles are labeled with a sentiment value (positive, negative, or neutral), which is based on natural language processing techniques.

In [12]:
from yahoo_fin import stock_info as si
import pandas as pd

In [13]:
start_date = '1970-02-01 00:00:00'
end_date = '2023-04-01 23:59:59'
ticker = "AAPL"
stock_data = si.get_data(ticker, start_date=start_date, end_date=end_date, interval='1d')

In [14]:
stock_data.shape

(10665, 7)

In [15]:
stock_data

,open,high,low,close,adjclose,volume,ticker
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600,AAPL
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800,AAPL
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000,AAPL
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600,AAPL
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600,AAPL
...,...,...,...,...,...,...,...
2023-03-27,159.940002,160.770004,157.869995,158.279999,158.279999,52390300,AAPL
2023-03-28,157.970001,158.490005,155.979996,157.649994,157.649994,45992200,AAPL
2023-03-29,159.369995,161.050003,159.350006,160.770004,160.770004,51305700,AAPL
2023-03-30,161.529999,162.470001,161.270004,162.360001,162.360001,49501700,AAPL


In [16]:
stock_df = pd.DataFrame(stock_data.to_records()).rename(columns={'index':'date'})
stock_df = stock_df.set_index("date")
stock_df.sort_index(ascending=True, inplace=True)

In [17]:
stock_df

,open,high,low,close,adjclose,volume,ticker
date,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600,AAPL
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800,AAPL
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000,AAPL
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600,AAPL
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600,AAPL
...,...,...,...,...,...,...,...
2023-03-27,159.940002,160.770004,157.869995,158.279999,158.279999,52390300,AAPL
2023-03-28,157.970001,158.490005,155.979996,157.649994,157.649994,45992200,AAPL
2023-03-29,159.369995,161.050003,159.350006,160.770004,160.770004,51305700,AAPL


In [18]:
# pip install newsapi-python

In [19]:
from newsapi import NewsApiClient
import csv
import os

def write_news(company_name,all_articles):

        if not os.path.exists("Compamy_data/"+company_name):
            os.mkdir("Compamy_data/"+company_name)

        for article in all_articles['articles']:
            title = article['title']
            source = article['source']['name']
            description = article['description']

            date_str = article['publishedAt'].split('T')[0]
        
            filename = f"Compamy_data/{company_name}/{date_str}.csv"

            if os.path.exists(filename):
                with open(filename, 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([title, source, description])
            else:
                # if the file doesn't exist, create it and write the headers
                with open(filename, 'w') as f:
                    writer = csv.writer(f)
                    writer.writerow(['Title', 'Source', 'Description'])
                    writer.writerow([title, source, description])

                
                
def get_and_write_news(ticker):
    
    company_name = yf.Ticker(ticker).info['longName']

    newsapi = NewsApiClient(api_key='7c7a335a8fa84c30908729aa8c2386be')
    
    all_articles = newsapi.get_everything(q=company_name,
                                          from_param='2023-03-04',
                                          to='2023-04-03',
                                          language='en',
                                          sort_by='relevancy',
                                          page=2)
    
    write_news(company_name,all_articles)


In [20]:
# pip install NewsApiClient

In [21]:
start_date = '1970-02-01 00:00:00'
end_date = '2023-04-01 23:59:59'

In [22]:
from tqdm import tqdm

tickers = si.tickers_sp500()


for ticker in tqdm(tickers):
    try: 
        stock_data = si.get_data(ticker, start_date=start_date, end_date=end_date, interval='1d')

        stock_df = pd.DataFrame(stock_data.to_records()).rename(columns={'index':'date'})
        stock_df = stock_df.set_index("date")
        stock_df.sort_index(ascending=True, inplace=True)
        path = "stock_datasets/df_{ticker}_jones.pkl"
        if not os.path.exists(path):
            os.makedirs(path)        
        stock_df.to_pickle(f"stock_datasets/df_{ticker}_jones.pkl")
        get_and_write_news(ticker)
    except:
        pass


100%|██████████| 503/503 [04:50<00:00,  1.73it/s]
